Kaan Karakaş 18070001042 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install openpyxl
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#read data
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/online_retail_II.xlsx',sheet_name='Year 2010-2011')

In [4]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
#For dropping invoices which start C
def prepare_retail(dataframe):
  dataframe.dropna(inplace=True)
  dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
  dataframe = dataframe[dataframe["Quantity"] > 0]
  dataframe = dataframe[dataframe["Price"] > 0]
  return dataframe

df = prepare_retail(data)

In [6]:
# Selecting just United Kingdom
uk_df = df[df['Country'] == 'United Kingdom']
uk_df.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
#For moving rows to columns
def create_apriori_datastructure(dataframe, id=False):    
    if id:
        grouped = uk_df.groupby(
            ['Invoice', 'StockCode'], as_index=False).agg({'Quantity': 'sum'})
        apriori_datastructure = pd.pivot(data=grouped, index='Invoice', columns='StockCode', values='Quantity').fillna(
            0).applymap(lambda x: 1 if x > 0 else 0)
        return apriori_datastructure
    else:
        grouped = uk_df.groupby(
            ['Invoice', 'Description'], as_index=False).agg({'Quantity': 'sum'})
        apriori_datastructure = pd.pivot(data=grouped, index='Invoice', columns='Description', values='Quantity').fillna(
            0).applymap(lambda x: 1 if x > 0 else 0)
        return apriori_datastructure

In [8]:
#Invoice product matrix
uk_apriori_df = create_apriori_datastructure(uk_df,True)
uk_apriori_df.head(5)

<ipython-input-7-5a8d6b667f49>:6: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  apriori_datastructure = pd.pivot(data=grouped, index='Invoice', columns='StockCode', values='Quantity').fillna(


StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
Invoice,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#We can see all possible combinations
def get_rules(apriori_df, min_support=0.01):
  frequent_itemsets = apriori(
      apriori_df, min_support=min_support, use_colnames=True)
  rules = association_rules(
      frequent_itemsets, metric="support", min_threshold=min_support)
  return rules

In [10]:
uk_rules = get_rules(uk_apriori_df)
uk_rules.sort_values(by='lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
506,(22916),(22917),0.010753,0.010873,0.010153,0.944134,86.829038,0.010036,17.705365
507,(22917),(22916),0.010873,0.010753,0.010153,0.933702,86.829038,0.010036,14.921137
511,(23171),(23170),0.013637,0.015980,0.011534,0.845815,52.930211,0.011316,6.382074
510,(23170),(23171),0.015980,0.013637,0.011534,0.721805,52.930211,0.011316,3.545575
483,(22746),(22745),0.012556,0.015619,0.010153,0.808612,51.769856,0.009956,5.143389
...,...,...,...,...,...,...,...,...,...
602,(85099B),(47566),0.086928,0.077496,0.010273,0.118176,1.524922,0.003536,1.046131
646,(85123A),(85099B),0.113601,0.086928,0.014538,0.127975,1.472195,0.004663,1.047071
647,(85099B),(85123A),0.086928,0.113601,0.014538,0.167243,1.472195,0.004663,1.064415
396,(85123A),(22423),0.113601,0.084705,0.013156,0.115812,1.367236,0.003534,1.035181


In [11]:
#It allows us to see the item name by looking at the stock code
def get_item_name(dataframe, stock_code):
  if type(stock_code) != list:
    product_name = dataframe[dataframe["StockCode"] ==
                             stock_code][["Description"]].values[0].tolist()
    return product_name
  else:
    product_names = [dataframe[dataframe["StockCode"]== product][[
        "Description"]].values[0].tolist()[0] for product in stock_code]
    return product_names

In [12]:
#It creates an output using the recomended product and get item name functions.
def get_output(target_id,dataframe,rules):
  target_product = get_item_name(dataframe,target_id)[0]
  recomended_product_ids = recomended_products(rules, target_id)
  recomended_product_names = get_item_name(dataframe,recomended_products(rules, target_id))
  print(f'Target Product ID (which is in the cart): {target_id}\nProduct Name: {target_product}')
  print(f'Recomended Product ID (which is in the cart): {recomended_product_ids}\nProduct Names: {recomended_product_names}')


In [13]:
#according to the table returns those linked to the given id
def recomended_products(rules_df, product_id, rec_count=5):
  sorted_rules = rules_df.sort_values('lift',ascending=False)
  recomended_products = []

  for i, product in sorted_rules["antecedents"].items():
    for j in list(product):
      if j == product_id:
        recomended_products.append(
            list(sorted_rules.iloc[i]["consequents"]))
  recomended_products = list({item for item_list in recomended_products for item in item_list})

  return recomended_products[:rec_count]

In [14]:
GET_PRODUCT_ID_1 =22916
GET_PRODUCT_ID_2 =22423
GET_PRODUCT_ID_3 =23171

In [15]:
get_item_name(uk_df, [GET_PRODUCT_ID_1,GET_PRODUCT_ID_2,GET_PRODUCT_ID_3 ])

['HERB MARKER THYME', 'REGENCY CAKESTAND 3 TIER', 'REGENCY TEA PLATE GREEN ']

In [16]:
get_output(GET_PRODUCT_ID_1,uk_df,uk_rules)

Target Product ID (which is in the cart): 22916
Product Name: HERB MARKER THYME
Recomended Product ID (which is in the cart): [21931]
Product Names: ['JUMBO STORAGE BAG SUKI']


In [17]:
get_output(GET_PRODUCT_ID_2,uk_df,uk_rules)

Target Product ID (which is in the cart): 22423
Product Name: REGENCY CAKESTAND 3 TIER
Recomended Product ID (which is in the cart): [22720, 23200, 23298, 23203, 23206]
Product Names: ['SET OF 3 CAKE TINS PANTRY DESIGN ', 'JUMBO BAG PEARS', 'SPOTTY BUNTING', 'JUMBO BAG DOILEY PATTERNS', 'LUNCH BAG APPLE DESIGN']


In [18]:
get_output(GET_PRODUCT_ID_3,uk_df,uk_rules)

Target Product ID (which is in the cart): 23171
Product Name: REGENCY TEA PLATE GREEN 
Recomended Product ID (which is in the cart): [22382, 20726]
Product Names: ['LUNCH BAG SPACEBOY DESIGN ', 'LUNCH BAG WOODLAND']
